In [2]:
import json, glob, os
import random, datetime

import logging
import pandas as pd
import csv

from jobspy import scrape_jobs


In [1]:
"""

ids are unique, global static

Agent Fisher
    - run tru all users, their settings, and their keywords
    - save the results in their corresponding folders

Agent Perry
`   
    - add new users and jobs. check if both are valid

    - ELT the data - remove duplicates, clean, etc
    - save the (agg)results in their corresponding folders

    - Generate Reports and Stats
    - sent emails, text messages?

Agent Krieger
    - run tru all jobs and apply to them


    LOGGING

    - folder structure does a lot for logging
    - log all errors and warnings - and the files that triggered them
    - saving files (advace state) is ~ permisive as of now.

    -----------

    - Log all errors and warnings
    - Log all successful runs
    - Log all failed runs

     [RUN-LOGS-(package)-E-Mail]

     encypted zip - stats.csv, sigma_quo.txt, 

     - polymorphic/other dp, encap, graph inherit


"""


class Agent_Krieger():
    pass




In [7]:

class Agent_Perry():

    """
    
    Perry is the Brain of the operation. He is responsible for the ELT of the data, and the 
    generation of reports and stats.
    
    for each user,
        for each date
            one dataframe with the columns found in the kw csv (standard) as well as date and 
            config folder columns to identify the source of the data


    # listing with the same title from the same company are considered duplicates, because
        # they are usually the same job offer, just posted multiple times for different locations
        jobs = jobs.drop_duplicates(subset=["title", "company"], keep="first")
        print(f"Number of (unique) jobs found: {len(jobs)}")

    """

    def __init__(self):
          
          print(" / ***Agent Perry is online *** /")
          print("Instantiating Perry...")
          self.user_configs = self.get_user_configs()
          self.perrys_todo = self.get_perrys_todo()
    
    # using the config files you can determine the users
    # then determine the dates for each user
    # then determine the search configs for each user

    @staticmethod # redundant, given it is a static method chekc later
    def get_user_configs() -> list[dict]: 

        user_config_paths = glob.glob('users/**/*.json')
        print(f" ---> Config file paths found : {user_config_paths}")
        user_configs = []

        for user_config_path in user_config_paths:
            with open(user_config_path) as f:
                tmp_dict = json.load(f)
                user_configs.append(tmp_dict)
        
        return user_configs


    def get_perrys_todo(self) -> dict:

        """
        This function helps with the ELT of the data. It returns a dictionary, which contains is 
        constructed to easily iterate over the data and apply the ELT process to it.

        """
        print(" ---> Getting Perry's To-Do List...")
        perrys_todo = {}

        for user_config in self.user_configs:
            username = user_config['user']

            try:
                #each search config folder we find has to be in this list, otherwise it can't be right.
                search_settings_user = [search_settings['name'] for search_settings in user_config['search_settings']]
                perrys_todo[username] = {}

            except KeyError:
                print(f"User: {username} - No search settings found - Key Error")
                continue

            if not search_settings_user:
                print(f"User: {username} - No search settings found - Empty List")
                continue

            
            dates = glob.glob(f'users/{username}/*')
            dates = [os.path.basename(date) for date in dates if date.endswith('.json') == False]  

            for date in dates:
                search_setting_folder_paths = glob.glob(f'users/{username}/{date}/*')
                perrys_todo[username][date] = search_setting_folder_paths


            # for each date in each user --->
            #   [username][date] = destination folder
            #   [username][date][search_setting_folderpath] = source folder(s)
            #   ... -> {'lucky': {'20240306': ['users/lucky/20240306/ger_py', 'users/lucky/20240306/lux_py']}}

            # Example:
            
            for user, scraper_runs in perrys_todo.items():
                
                print(f"\n --- Destination folders for {user}: ---")

                for date_entry, search_setting_folders in scraper_runs.items():
                    destination_folder = os.path.join('users', user, date_entry)
                    print(f"{destination_folder}")
                    
                    for search_setting_folder in search_setting_folders:
                        search_name = os.path.basename(search_setting_folder)

                        # check if search setting folder found by glob is actually in the user config
                        if search_name in search_settings_user:
                            print(f"    - {os.path.basename(search_setting_folder)} --- OK")
                        else:
                            print(f"    - {os.path.basename(search_setting_folder)} --- NOT FOUND")
                            return 0

        return perrys_todo


    def filter_jobs(jobs: pd.DataFrame, search_setting: dict) -> pd.DataFrame:
            pass
    

    def send_logs() -> None:
        pass

    def send_email() -> None:
        pass


agent_perry = Agent_Perry()
todo = agent_perry.perrys_todo
print(todo)



 / ***Agent Perry is online *** /
Instantiating Perry...
 ---> Config file paths found : ['users/tdawg/tdawg_config.json', 'users/lucky/lucky_config.json']
 ---> Getting Perry's To-Do List...
User: tdawg - No search settings found - Key Error

 --- Destination folders for lucky: ---
users/lucky/20240306
    - ger_py --- OK
    - lux_py --- OK
{'lucky': {'20240306': ['users/lucky/20240306/ger_py', 'users/lucky/20240306/lux_py']}}


In [34]:

class Agent_Fisher():
    def __init__(self):

        """
        Agent Fisher is the muscle of the operation. It is responsible for running the scrapes.

        get_user_configs - returns a list of dictionaries, each dictionary is a user config.
        get_proxy - returns a string of the proxy to be passed to the scraper as a string.
        get_date - returns a string of the date in the format YYYYMMDD.
        Date, user and search_settings(stored in user_configs - eg. IT jobs in london, 
        remote teaching jobs in Germany). 
        """
        print(" / ***Agent Fisher is online *** /")
        print("Instantiating Fisher...")
        self.date_run = self.get_date()
        self.proxy = self.get_proxy()
        self.user_configs = self.get_user_configs()

    # --- INIT Functions --- #

    @staticmethod
    def get_date() -> str:
        now = datetime.datetime.now()
        date = now.strftime("%Y%m%d")
        
        return date
    
    @staticmethod
    def get_user_configs() -> list[dict]: 

        user_config_paths = glob.glob('users/**/*.json')
        print(f" ---> Config file paths found : {user_config_paths}")
        user_configs = []

        for user_config_path in user_config_paths:
            with open(user_config_path) as f:
                tmp_dict = json.load(f)
                user_configs.append(tmp_dict)
        
        return user_configs

    @staticmethod # proxy path is hardcoded
    def get_proxy() -> str:
        
        proxy_path = os.path.join("results", "proxy.txt")
        with open(proxy_path, "r") as f:
            proxy = f.read().strip()

        return proxy


    # --- SEARCH --- #

    def update_keywords_left(self, kewords_for_this_search: list, ss_path: str) -> list[str]:

        """
        This function checks which keywords have already been run and determines the state of the run.
        If a keyword has already been run (clean, no exception thrown) it is removed from the list (state).
        Exceptions are there to handle the case where the run was not completed, incomplete or the proxy
        was blocked.

        ss_path: str - user/date/search_setting - path to the search setting folder

        Note - the order of the keywods run is picked randomly. To spice things up a bit ...
        """
        
        files = glob.glob(f"{ss_path}/*.csv") ### os path join
        files = [os.path.basename(file) for file in files]

        keywords_done = [keyword for keyword in kewords_for_this_search if f"{keyword}.csv" in files]
        keywords_left = [keyword for keyword in kewords_for_this_search if keyword not in keywords_done]

        print(f"--- Path: {ss_path}/*.csv ---")
        print(f"Keywords done: {len(keywords_done)} : {keywords_done}")
        print(f"Keywords left: {len(keywords_left)} : {keywords_left}")
        return keywords_left


    def run_search_setting(self, username: str, search_setting: dict, date: str) -> bool:
            
        empty_jobs = pd.DataFrame(columns=[ "job_url",
        "site", "title", "company", "company_url", "location", "job_type",
        "date_posted", "interval", "min_amount", "max_amount", "currency",
        "is_remote", "num_urgent_words", "benefits", "emails", "description"])

        keywords = search_setting['keywords']
        jobs = empty_jobs

        ss_path = os.path.join("users",username, date, search_setting['name'])
        keywords_left = self.update_keywords_left(keywords, ss_path)

        # ---- Iterate over keywords ---- #
        while keywords_left:

            keyword = random.choice(keywords_left)
            kw_path = os.path.join(ss_path, f"{keyword}.csv")

            print(f"Keyword: *** {keyword} *** -> Starting search...")

            try:
                jobs = scrape_jobs(
                    site_name=search_setting['site_name'],

                    search_term=keyword,
                    proxy=self.proxy,

                    hours_old=search_setting['hours_old'],
                    is_remote=search_setting['is_remote'],
                    results_wanted=search_setting['results_wanted'],
                    country_indeed=search_setting['country_indeed']  # only needed for indeed / glassdoor
                )

            except Exception as e:
                print(f"Error with keyword: {keyword}")
                print(f"{e}")
                if "Bad proxy" in str(e):
                    # if the proxy is not working, the program will never advance, so we stop it.
                    print("Bad proxy, stopping the program")

                    " TODO: Outbound email to notify the admin, maybe change proxy"
                    return False
                
                elif "Could not find any results for the search" in str(e):
                    # if this happens, we write an empty file and continue, because the prog does not advance
                    # NOTE : even tho this error is thrown, searching indeed for the kw often returns results
                    print(f"No jobs found for keyword: {keyword}. Writing empty file...")
                    empty_jobs.to_csv(kw_path, quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False)
                    keywords_left = self.update_keywords_left(keywords, ss_path)
                
                continue

                #2024-03-06 13:36:15,683 - JobSpy - ERROR - Indeed: failed to do request: Get "https://lu.indeed.com/m/jobs?q=openai&l=luxembourg&filter=0&start=70&sort=date&fromage=3": http: server gave HTTP response to HTTPS client
                #HTTPSConnectionPool(host='apis.indeed.com', port=443): Max retries exceeded with url: /graphql (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1007)')))
                #Error with keyword: openai Could not find any results for the search. /graphql (Caused by ProxyError('Unable to connect to proxy', 

                # Not all exceptions are critical, and for most we can try again
                # this is why we continue and not save the empty file
            
            print(f"Number of jobs found: {len(jobs)}")
            jobs = jobs.drop_duplicates(subset=["job_url"], keep="first")
            
            if jobs.empty:
                print(f"No jobs found for keyword: {keyword}. Writing empty file...")
                empty_jobs.to_csv(kw_path, quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False)
                
            else:
                print(f"Writing jobs to file: {kw_path}")
                jobs.to_csv(kw_path, quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False)
                jobs = empty_jobs

            keywords_left = self.update_keywords_left(keywords, ss_path)
        # --- End of while loop --- #
        
        return True

    # --- Overarching --- #

    def run_user(self, user_config: dict) -> None:
        
        username = user_config['user']
        print(f"Running user: {username}")
        
        successful_run = []

        try:
            search_settings = user_config['search_settings']
        except KeyError:
            print(f"User: {username} - No search settings found - Key Error")
            return

        if not search_settings:
            print(f"User: {username} - No search settings found - Empty List")
            return

        for search_setting in user_config['search_settings']:

            # try creqting the folder
            date_path = os.path.join("users", username, self.date_run)
            search_setting_path = os.path.join(date_path, search_setting['name'])

            #if folders already exist, we skip the creation. Take this into account for Perry
            if not os.path.exists(date_path):
                os.makedirs(date_path)
                print(f"Created folder: {date_path}")
            if not os.path.exists(search_setting_path):
                os.makedirs(search_setting_path)
                print(f"Created folder: {search_setting_path}")

            print(f"Running search setting: {search_setting['name']} for user: {username}")
            successful_run.append(self.run_search_setting(username, search_setting, self.date_run))

        
        if all(successful_run):
            print(f"User: {username} - All search settings ran successfully")
        else:
            print(f"User: {username} - Some search settings failed. Run is incomplete")


    def run_all_users(self):

        #date = self.date_run
        #user_configs = self.user_configs


        #print(f" ---> Date : {date}")
        pass

agent_fish = Agent_Fisher()
agent_fish.run_user(agent_fish.user_configs[1])





 ---> Config file paths found : ['users/tdawg/tdawg_config.json', 'users/lucky/lucky_config.json']
Running user: lucky
Running search setting: lux_py for user: lucky
--- Path: users/lucky/20240306/lux_py/*.csv ---
Keywords done: 14 : ['python', 'database', 'nlp', 'natural language', 'signal processing', 'prompt engineer', 'openai', 'language model', 'generative model', 'genai', 'ki', 'künstliche intelligenz', 'ai', 'artificial intelligence']
Keywords left: 0 : []
Running search setting: ger_py for user: lucky
--- Path: users/lucky/20240306/ger_py/*.csv ---
Keywords done: 14 : ['python', 'database', 'nlp', 'natural language', 'signal processing', 'prompt engineer', 'openai', 'language model', 'generative model', 'genai', 'ki', 'künstliche intelligenz', 'ai', 'artificial intelligence']
Keywords left: 0 : []
User: lucky - All search settings ran successfully
